# Part One: Scrape NASA
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables to reference later.

In [3]:
#Import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import time

In [6]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
db = client.mars_db
collection = db.news
mars_news = db.news.find()


In [10]:

url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')


In [13]:
headlines = soup.find_all('div', class_='content_title')
latest_headline = headlines[0].text
latest_headline = latest_headline.strip()
latest_headline

"NASA Readies Perseverance Mars Rover's Earthly Twin"

In [19]:

article = soup.find_all('div', class_='image_and_description_container')
# paragraph

latest_paragraph = article[0]
latest_paragraph = latest_paragraph.find('div', class_='rollover_description_inner')
latest_paragraph = latest_paragraph.text.strip()
latest_paragraph


"Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."

In [22]:
mars_news_dict = {
    "latest_headline": latest_headline,
    "latest_paragraph": latest_paragraph
}

db.news.insert_one(mars_news_dict)

# Part Two: Scrape JPL for featured image
Visit the url for JPL Featured Space Image here. 
Use splinter to navigate the site and 
Find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [27]:
executable_path = {'executable_path': r'C:\Users\diamo\Bootcamp\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [50]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)
browser.click_link_by_partial_text('more info')

C:\Users\diamo\anaconda3.7\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [51]:
html = browser.html
soup = bs(html, 'html.parser')
image_url = soup.find('figure', class_='lede').a['href']
image_url

'/spaceimages/images/largesize/PIA16217_hires.jpg'

In [33]:
base_url = 'https://www.jpl.nasa.gov'
full_image_url = base_url + image_url
full_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16227_hires.jpg'

In [35]:
image = {
    "full_image_url": full_image_url
}
#Put into db w/its own collection? 
db.image.insert_one(image)

# Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [49]:
mars_url = 'https://space-facts.com/mars/'
mars_facts = pd.read_html(mars_url)[0]
# mars_facts
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

# Part 4: Mars Hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mars' hemispheres.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere

* Cerberus
* Valles Marineris
* Schiaparelli 
* Syrtis Major


In [53]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
response = requests.get(hemisphere_url)
soup = bs(response.text, 'lxml')


In [54]:
cerberus_title = soup.find('title').text
cerberus_title = cerberus_title.split('|')
cerberus_title = cerberus_title[0]
cerberus_title

'Cerberus Hemisphere Enhanced '

In [55]:
cerberus_hemisphere = soup.find_all('div', class_= 'wide-image-wrapper')
cerberus_hemisphere = cerberus_hemisphere[0]
cerberus_hemisphere = cerberus_hemisphere.find_all('img', class_='wide-image')
cerberus_hemisphere = cerberus_hemisphere[0]
cerberus_hemisphere = cerberus_hemisphere['src']
# cerberus_hemisphere

'/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [58]:
base_url = 'https://astrogeology.usgs.gov/'
cerberus_full_url = base_url + cerberus_hemisphere
# cerberus_full_url
hemisphere_image_urls = [
    {"title": cerberus_title, "img_url": cerberus_full_url}
]

In [59]:
url2 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
response = requests.get(url2)
soup = bs(response.text, 'lxml')

In [60]:
valles_title = soup.find('title').text
valles_title = valles_title.split('|')
valles_title = valles_title[0]
# valles_title

In [61]:
valles_hemisphere = soup.find_all('div', class_= 'wide-image-wrapper')
valles_hemisphere = valles_hemisphere[0]
valles_hemisphere = valles_hemisphere.find_all('img', class_='wide-image')
valles_hemisphere = valles_hemisphere[0]
valles_hemisphere = valles_hemisphere['src']
# valles_hemisphere

In [62]:
valles_full_url = base_url + valles_hemisphere
# valles_full_url
hemisphere_image_urls.append({"title": valles_title, "img_url": valles_full_url})


[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [63]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [64]:
url3 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'
response = requests.get(url3)
soup = bs(response.text, 'lxml')

In [65]:
schiaparelli_title = soup.find('title').text
schiaparelli_title = schiaparelli_title.split('|')
schiaparelli_title = schiaparelli_title[0]
# schiaparelli_title

'Schiaparelli Hemisphere Enhanced '

In [67]:
schiaparelli_hemisphere = soup.find_all('div', class_= 'wide-image-wrapper')
schiaparelli_hemisphere = schiaparelli_hemisphere[0]
schiaparelli_hemisphere = schiaparelli_hemisphere.find_all('img', class_='wide-image')
schiaparelli_hemisphere = schiaparelli_hemisphere[0]
schiaparelli_hemisphere = schiaparelli_hemisphere['src']
# schiaparelli_hemisphere

In [69]:
schiaparelli_full_url = base_url + schiaparelli_hemisphere
# schiaparelli_full_url
hemisphere_image_urls.append({"title": schiaparelli_title, "img_url": schiaparelli_full_url})


In [70]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}]

In [71]:
url4 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'
response = requests.get(url4)
soup = bs(response.text, 'lxml')

In [72]:
syrtis_title = soup.find('title').text
syrtis_title = syrtis_title.split('|')
syrtis_title = syrtis_title[0]
# syrtis_title

'Syrtis Major Hemisphere Enhanced '

In [74]:
syrtis_hemisphere = soup.find_all('div', class_= 'wide-image-wrapper')
syrtis_hemisphere = syrtis_hemisphere[0]
syrtis_hemisphere = syrtis_hemisphere.find_all('img', class_='wide-image')
syrtis_hemisphere = syrtis_hemisphere[0]
syrtis_hemisphere = syrtis_hemisphere['src']
# syrtis_hemisphere

In [75]:
syrtis_full_url = base_url + syrtis_hemisphere
syrtis_full_url
hemisphere_image_urls.append({"title": syrtis_title, "img_url": syrtis_full_url})


In [76]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced ',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}]